# 導入

# 参考文献

1. [OpenCVのCanny関数によるエッジ検出について](http://labs.eecs.tottori-u.ac.jp/sd/Member/oyamada/OpenCV/html/py_tutorials/py_imgproc/py_canny/py_canny.html)
2. [Flipについて](https://qiita.com/yu4u/items/855ff350e6d93c82afd5)

# 目次

* [<font size=4>EDA</font>](#1)
    * [データの下準備](#1.1)
    * [サンプル画像の読み込み](#1.2)
    * [チャネル値調査](#1.3)
    * [葉っぱの画像の読み込み](#1.4)
    * [その他EDA](#1.5)


* [<font size=4>画像処理</font>](#2)
    * [Canny関数によるエッジ検出](#2.1)
    * [Flipping（画像の回転と反転）](#2.2)
    * [Convolution(畳み込み)](#2.3)
    * [Blurring(ぼやけた画像)](#2.4)
  

* [<font size=4>モデル構築</font>](#3)
    * [データの下準備](#3.1)
    * [DenseNet](#3.2)
    * [EfficientNet](#3.3)

* [<font size=4>まとめと考察</font>](#4)

# EDA <a id="1"></a>

## データの下準備 <a id="1.1"></a>

### 必要なライブラリのインストール

画像認識モデルはkaggleではdensenetとefficientnetをよく使っているカーネルが多かったのでこの2つを使ってみる

In [ ]:
# efficientnetのinstall
!pip install -q efficientnet

In [ ]:
# 基本

## ターミナルの記述などを行える。
import os

## numpy pandas
import numpy as np
import pandas as pd

## 画像解析用のライブラリ
import cv2

## 可視化
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm #完了までのバーを表示してくれる

tqdm.pandas()
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

np.random.seed(0)

## ディープラーニング
from sklearn.model_selection import train_test_split
import tensorflow as tf
from IPython.display import SVG
import efficientnet.tfkeras as efn
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model
from kaggle_datasets import KaggleDatasets #ダウンロードしなくてもaggleのデータにアクセスできるようにする
from tensorflow.keras.applications import DenseNet121

import warnings
warnings.filterwarnings("ignore")

### データのpathの設定

In [ ]:
EPOCHS = 20 #エポック数を20に設定

SAMPLE_LEN = 100 #適当に画像を選別する際など

## ここで設定したpathを使って画像データやテストデータにアクセスする
image_path = "../input/plant-pathology-2020-fgvc7/images/"
test_path = "../input/plant-pathology-2020-fgvc7/test.csv"
train_path = "../input/plant-pathology-2020-fgvc7/train.csv"
submission_path = "../input/plant-pathology-2020-fgvc7/sample_submission.csv"

test = pd.read_csv(test_path) #テストデータ
train = pd.read_csv(train_path) #訓練データ

## pathがちゃんと通っているか確認
train.head()

trainデータが読み込めていたので成功！

### 適当な一枚を読み込む

In [ ]:
## trainデータは読み込めていたのでフォルダ内の画像を認識できる形で読み込めるか調べる

## 画像読み込みの関数
def load_image(image_id):
    file_path = image_id + ".jpg"
    image = cv2.imread(image_path + file_path)
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# 100枚をランダムにimagesフォルダから取得
train_images = train["image_id"][:SAMPLE_LEN].progress_apply(load_image)

## サンプル画像の読み込み <a id="1.2"></a>

In [ ]:
leaf_fig = px.imshow(cv2.resize(train_images[42], (200, 150))) #サイズを調整した上でcv2を使って読み込み
leaf_fig.show()

読み込みは成功している！

In [ ]:
## さらに適当な1枚をピックアップ
leaf_fig_2 = px.imshow(cv2.resize(train_images[1],(450, 300))) #サイズを変更してみる
leaf_fig_2.show()

こちらは別の病状の葉の画像。読み込みは問題なさそうなのであとはどうやって病気を見分けていくかを考える

多くのカーネルで、channel distributionsというrgbで画像を見た際の赤、緑、青の割合を調べていたので真似してみる。

健康な葉っぱは緑の割合が多く、rust（さび）やmultiple diseases（複合）は赤の割合が多いなどありそう。

## チャネル値調査 <a id="1.3"></a>

In [ ]:
# 1つ目の次元が赤（[:,:,0]）
red_values = [np.mean(train_images[idx][:,:,0]) for idx in range(len(train_images))] #取得した100枚全部の赤分布の割合の平均値を取得

# 2つ目の次元が緑（[:,:,1]）
green_values = [np.mean(train_images[idx][:,:,1]) for idx in range(len(train_images))]
# 3つ目の次元が青（[:,:,2]）
blue_values = [np.mean(train_images[idx][:,:,2]) for idx in range(len(train_images))]

# 色彩全体の平均
values = [np.mean(train_images[idx]) for idx in range(len(train_images))]

### RGB全てのチャネル

In [ ]:
fig = ff.create_distplot([values], group_labels=["Channel全体"], colors=["purple"])
fig.update_layout(showlegend=False, template="simple_white")
fig.update_layout(title_text="チャネル全体の分布")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.5
fig

### 赤の分布

In [ ]:
fig = ff.create_distplot([red_values], group_labels=["R"], colors=["red"])
fig.update_layout(showlegend=False, template="simple_white")
fig.update_layout(title_text="赤の分布")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.5
fig

全体がおおよそな正規分布だったのに対して、赤は右に歪みがある。つまり、あまりメインの色ではない。

### 緑の分布

In [ ]:
fig = ff.create_distplot([green_values], group_labels=["G"], colors=["green"])
fig.update_layout(showlegend=False, template="simple_white")
fig.update_layout(title_text="緑の分布")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.5
fig

葉の画像なので当たり前だが赤よりも分布が多い。

### 青の分布

In [ ]:
fig = ff.create_distplot([blue_values], group_labels=["B"], colors=["blue"])
fig.update_layout(showlegend=False, template="simple_white")
fig.update_layout(title_text="青の分布")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.5
fig

In [ ]:
### 3色の分布を箱ヒゲ図で見てみる
one_leaf = go.Figure()

for idx, values in enumerate([red_values, green_values, blue_values]):
    if idx == 0:
        color = "Red"
    if idx == 1:
        color = "Green"
    if idx == 2:
        color = "Blue"
    one_leaf.add_trace(go.Box(x=[color]*len(values), y=values, name=color, marker=dict(color=color.lower())))

one_leaf.update_layout(yaxis_title="平均値", xaxis_title="色", title="色ごと平均値", template="plotly_white")

- 255のrgbの振り分けで見ると緑が一番多く、ついで赤、青という順。
- 病気によってはここの割合が違ったりしそう

In [ ]:
## チャネル値を1つのグラフで可視化

fig = ff.create_distplot([red_values, green_values, blue_values],group_labels=["R", "G", "B"],colors=["red", "green", "blue"])
fig.update_layout(title_text="Distribution all channel values", template="simple_white")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.5
fig.data[1].marker.line.color = 'rgb(0, 0, 0)'
fig.data[1].marker.line.width = 0.5
fig.data[2].marker.line.color = 'rgb(0, 0, 0)'
fig.data[2].marker.line.width = 0.5
fig

### 今回見分ける病気
- healthy
- multiple_diseases
- rust
- scab

これらのrgbの割合を学習させれば良さそう

## 葉っぱの画像の読み込み <a id="1.4"></a>

In [ ]:
## 画像読み込みの関数を作成
### 状態の名称をtrainデータのカラムから取得し、それを指定して画像を読み込ませる仕組みにする
def visualize_leaves(cond=[0, 0, 0, 0], cond_cols=["healthy"], is_cond=True):
    if not is_cond:
        cols, rows = 3, min([3, len(train_images)//3])
        fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(30, rows*20/3))
        for col in range(cols):
            for row in range(rows):
                ax[row, col].imshow(train_images.loc[train_images.index[-row*3-col-1]])
        return None
        
    cond_0 = "healthy == {}".format(cond[0])
    cond_1 = "scab == {}".format(cond[1])
    cond_2 = "rust == {}".format(cond[2])
    cond_3 = "multiple_diseases == {}".format(cond[3])
    
    cond_list = []
    for col in cond_cols:
        if col == "healthy":
            cond_list.append(cond_0)
        if col == "scab":
            cond_list.append(cond_1)
        if col == "rust":
            cond_list.append(cond_2)
        if col == "multiple_diseases":
            cond_list.append(cond_3)
    
    data = train.loc[:99]
    for cond in cond_list:
        data = data.query(cond)
        
    images = train_images.loc[list(data.index)]
    cols, rows = 3, min([3, len(images)//3])
    
    fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(30, rows*20/3))
    for col in range(cols):
        for row in range(rows):
            ax[row, col].imshow(images.loc[images.index[row*3+col]])
    plt.show()

### Healthy

In [ ]:
visualize_leaves(cond=[1, 0, 0, 0], cond_cols=["healthy"])

斑点や赤くなってしまっている箇所がほぼない、緑の割合が多い画像

### scab

In [ ]:
visualize_leaves(cond=[0, 1, 0, 0], cond_cols=["scab"])

葉のいくつかの箇所に黒ずんでしまっている箇所が存在する

### rust

In [ ]:
visualize_leaves(cond=[0, 0, 1, 0], cond_cols=["rust"])

赤、黄色の斑点がいくつかの箇所に存在する。rustの画像は赤の割合が他より高くなっていそう

### multiple diseases

In [ ]:
visualize_leaves(cond=[0, 0, 0, 1], cond_cols=["multiple_diseases"])

複合なのでrustかつscabなども見られる。これらをどう見分けるか

## その他EDA <a id="1.5"></a>

### 円グラフ

In [ ]:
## 円グラフでtrainデータの葉っぱの割合分布を見てみる
fig = go.Figure([go.Pie(labels=train.columns[1:], values=train.iloc[:,1:].sum().values)])
fig.update_layout(title_text="状態分布", template="simple_white")
fig.data[0].marker.line.color = 'rgb(0,0,0)'
fig.data[0].marker.line.width = 0.5
fig.show()

- multipleの割合が少なく学習に影響があるかも
- それ以外のものはおおよそ均等に画像データが与えられている

## 棒グラフ

In [ ]:
## 棒グラフでtrainデータの葉っぱの割合分布を見てみる
fig = go.Figure([go.Bar(x=train.columns[1:], y=train.iloc[:,1:].sum().values)])
fig.update_layout(title_text="状態分布", template="simple_white")
fig.data[0].marker.line.color = 'rgb(0,0,0)'
fig.data[0].marker.line.width = 0.5
fig.show()

- healthy, rust, scabは良いがmultiple_diseasesは枚数が少ないため水増しをしたい
- 多くのカーネルで行われていた転置などを行うのが良さそう

# 画像処理 <a id="2"></a>

## Canny関数によるエッジ検出 <a id="2.1"></a>

多くのカーネルで「エッジ検出」という明るさ変化が大きい点を抽出する処理を行っているため、参考文献の記事を真似しながらエッジ検出を行ってみる。

エッジ検出を行うことで、画像の中のどの部分が葉っぱなのかをAIが理解しやすくなり、モデルの精度が上がるらしい。

ここではOpenCVのCanny関数を使用。

### Cannyを使ったエッジ検出に関する参考文献を一部抜粋
1. ノイズ削減
エッジ検出は画像中のノイズに対して敏感なため，まず初めに画像を平滑化してノイズを削減します．具体的には5x5のサイズの Gaussianフィルタを使います．フィルタリングの方法は既に前のチュートリアルで見ているかと思います．

2. 画像の輝度勾配を見つける
次に，平滑化された画像からSobelフィルタを使って縦方向(G_y)と横方向(G_x)の1次微分を取得します．これら2つの微分画像から以下のようにエッジの勾配と方向を求めます:

$$
Edge\_Gradient \; (G) = \sqrt{G_x^2 + G_y^2}
$$

$$
Angle \; (\theta) = \tan^{-1} \bigg(\frac{G_y}{G_x}\bigg)
$$

勾配方向は常にエッジに対して直交します．勾配方向は横，縦，二つの対角方向の内どれか一つになります．

3. 非極大値の抑制
勾配の方向と強度を計算した後は，エッジと関係ない画素を取り除きます．具体的には，各画素に対してその画素が勾配方向に対して極大値であるかどうかを確認します

4. ヒステリシス(Hysteresis)を使ったしきい値処理
前処理で検出されたエッジの内，正しいエッジとそうでないものを区別します．この区別をするために， minVal と maxVal という二つのしきい値を使います．画素値の微分値が maxVal 以上であれば正しいエッジとみなし， minVal 以下の値であればエッジではないとみなし除外します．微分値が二つのしきい値の間であれば，正しいエッジとそうでないエッジとの隣接関係を基に区別します．正しいエッジと区別された画素につながっていれば正しいエッジとみなし，そうでなければエッジではない画素とみなします

この処理では，エッジは長い線であるという前提のもと，少数の画素で構成されるエッジも削除します．

最終的に画像中の強いエッジを検出できます．

他のカーネルを参考にしながらエッジ検出の関数を記述する。

In [ ]:
## エッジ検出の関数
def edge_and_cut(img):
    emb_img = img.copy()
    edges = cv2.Canny(img, 100, 200)
    edge_coors = []
    for i in range(edges.shape[0]):
        for j in range(edges.shape[1]):
            if edges[i][j] != 0:
                edge_coors.append((i, j))
    
    row_min = edge_coors[np.argsort([coor[0] for coor in edge_coors])[0]][0]
    row_max = edge_coors[np.argsort([coor[0] for coor in edge_coors])[-1]][0]
    col_min = edge_coors[np.argsort([coor[1] for coor in edge_coors])[0]][1]
    col_max = edge_coors[np.argsort([coor[1] for coor in edge_coors])[-1]][1]
    new_img = img[row_min:row_max, col_min:col_max]
    
    emb_img[row_min-10:row_min+10, col_min:col_max] = [255, 0, 0]
    emb_img[row_max-10:row_max+10, col_min:col_max] = [255, 0, 0]
    emb_img[row_min:row_max, col_min-10:col_min+10] = [255, 0, 0]
    emb_img[row_min:row_max, col_max-10:col_max+10] = [255, 0, 0]
    
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(30, 20))
    ax[0].imshow(img, cmap='gray')
    ax[0].set_title('original', fontsize=24)
    ax[1].imshow(edges, cmap='gray')
    ax[1].set_title('canny edge', fontsize=24)
    ax[2].imshow(emb_img, cmap='gray')
    ax[2].set_title('trimming', fontsize=24)
    plt.show()

In [ ]:
edge_and_cut(train_images[42])
edge_and_cut(train_images[25])
edge_and_cut(train_images[31])

- エッジ検出によって葉の輪郭を光の勾配が急に変化する箇所を見分け、画像の中のどの部分が葉なのかを検出。
- 赤枠の中を主に分析し、画像の病状を分類する

## Flipping（画像の回転と反転） <a id="2.2"></a>

ここから先は画像の水増しを3パターンによって行う。まずは回転と反転

In [ ]:
def invert(img):
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(30, 20))
    ax[0].imshow(img)
    ax[0].set_title('original', fontsize=24)
    ax[1].imshow(cv2.flip(img, 0))
    ax[1].set_title('upside down', fontsize=24)
    ax[2].imshow(cv2.flip(img, 1))
    ax[2].set_title('horizontal flip', fontsize=24)
    plt.show()

In [ ]:
invert(train_images[42])
invert(train_images[25])
invert(train_images[31])

- 上下反転、左右反転した状態の画像に対しても学習を行うことで汎用性の高いモデルを作成する

## Convolution(畳み込み)<a id="2.3"></a>

In [ ]:
def conv(img):
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 20))
    kernel = np.ones((7, 7), np.float32)/25
    conv = cv2.filter2D(img, -1, kernel)
    ax[0].imshow(img)
    ax[0].set_title('original', fontsize=24)
    ax[1].imshow(conv)
    ax[1].set_title('convolved', fontsize=24)
    plt.show()

In [ ]:
conv(train_images[42])
conv(train_images[25])
conv(train_images[31])

こちらも水増しの役割。日光を浴びているような画像になった

## Blurring(ぼやけた画像) <a id="2.4"></a>

元の画像をぼやけさせるblur関数を使用

In [ ]:
def blur(img):
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 20))
    ax[0].imshow(img)
    ax[0].set_title('original', fontsize=24)
    ax[1].imshow(cv2.blur(img, (100, 100)))
    ax[1].set_title('blurred', fontsize=24)
    plt.show()

In [ ]:
blur(train_images[42])
blur(train_images[25])
blur(train_images[31])

想像以上にぼやけたが、rustなどは特徴が掴めていそうなので学習は出来そう

# モデル構築 <a id="3"></a>

## データの下準備 <a id="3.1"></a>

今回は多くのカーネルで使用されていたDenseNetとEfficientNetの2つを作成し、精度を比べてみる。

### TPUの下準備
kaggleのカーネルで使われているTRUを使用するための下準備を真似る。（notebookのAcceleratorはTPUv3-8に変更済み）

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

BATCH_SIZE = 16 * strategy.num_replicas_in_sync
GCS_DS_PATH = KaggleDatasets().get_gcs_path()

### 病気のラベルとpathの読み込み

In [ ]:
def format_path(st):
    return GCS_DS_PATH + '/images/' + st + '.jpg'

test_paths = test.image_id.apply(format_path).values
train_paths = train.image_id.apply(format_path).values

train_labels = np.float32(train.loc[:, 'healthy':'scab'].values)
train_paths, valid_paths, train_labels, valid_labels =\
train_test_split(train_paths, train_labels, test_size=0.15, random_state=2020)

In [ ]:
def decode_image(filename, label=None, image_size=(512, 512)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    
    if label is None:
        return image
    else:
        return image, label

def data_augment(image, label=None):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    if label is None:
        return image
    else:
        return image, label

### データセットのオブジェクトを作成

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_paths, train_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .map(data_augment, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(512)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((valid_paths, valid_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(test_paths)
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
)

### 追加機能を設定

In [ ]:
def build_lrfn(lr_start=0.00001, lr_max=0.00005, 
               lr_min=0.00001, lr_rampup_epochs=5, 
               lr_sustain_epochs=0, lr_exp_decay=.8):
    lr_max = lr_max * strategy.num_replicas_in_sync

    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) *\
                 lr_exp_decay**(epoch - lr_rampup_epochs\
                                - lr_sustain_epochs) + lr_min
        return lr
    return lrfn

### ハイパーパラメータとコールバックの設定

In [ ]:
lrfn = build_lrfn()
STEPS_PER_EPOCH = train_labels.shape[0] // BATCH_SIZE
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)

正直この辺まではTPUのコピペ。この先のdensenetとefficientnetの2つを使って精度の違いを検証

## DenseNet <a id="3.2"></a>

In [ ]:
with strategy.scope():
    model = tf.keras.Sequential([DenseNet121(input_shape=(512, 512, 3),
                                             weights='imagenet',
                                             include_top=False),
                                 L.GlobalAveragePooling2D(),
                                 L.Dense(train_labels.shape[1],
                                         activation='softmax')])
        
    model.compile(optimizer='adam',
                  loss = 'categorical_crossentropy',
                  metrics=['categorical_accuracy'])
    model.summary()

### DenseNet 基本ブロック

In [ ]:
SVG(tf.keras.utils.model_to_dot(Model(model.layers[0].input, model.layers[0].layers[13].output), dpi=70).create(prog='dot', format='svg'))

DenseNetの基本ブロックの構造を図解。

### 使うモデルの図解

参考にしたカーネルで使われていたDenseNetからpoolingへと続くモデルを真似して構築してみる

In [ ]:
SVG(tf.keras.utils.model_to_dot(model, dpi=70).create(prog='dot', format='svg'))

### 訓練データの読み込み

In [ ]:
history = model.fit(train_dataset,
                    epochs=EPOCHS,
                    callbacks=[lr_schedule],
                    steps_per_epoch=STEPS_PER_EPOCH,
                    validation_data=valid_dataset)

### 学習結果を取得

In [ ]:
## DenseNetのエポックごとのLossと正解率を可視化
def display_training_curves(training, validation, yaxis):
    if yaxis == "loss":
        ylabel = "Loss"
        title = "エポックごとのloss"
    else:
        ylabel = "Accuracy"
        title = "エポックごとの正解率"
        
    fig = go.Figure()
        
    fig.add_trace(
        go.Scatter(x=np.arange(1, EPOCHS+1), mode='lines+markers', y=training, marker=dict(color="dodgerblue"),
               name="Train"))
    
    fig.add_trace(
        go.Scatter(x=np.arange(1, EPOCHS+1), mode='lines+markers', y=validation, marker=dict(color="darkorange"),
               name="Val"))
    
    fig.update_layout(title_text=title, yaxis_title=ylabel, xaxis_title="Epochs", template="plotly_white")
    fig.show()

### グラフ

In [ ]:
display_training_curves(
    history.history['categorical_accuracy'], 
    history.history['val_categorical_accuracy'], 
    'accuracy')

11エポックあたりから正解率は90%を超えている。最初の方はtrainとvalidationの正解率にバラツキがあるが一定以上学習が進むとどちらも高い正解率を示す。

### DenseNetの分類結果をいくつか表示

In [ ]:
def process(img):
    return cv2.resize(img/255.0, (512, 512)).reshape(-1, 512, 512, 3)
def predict(img):
    return model.layers[2](model.layers[1](model.layers[0](process(img)))).numpy()[0]

fig = make_subplots(rows=4, cols=2)
preds = predict(train_images[2])

colors = {"Healthy":px.colors.qualitative.Plotly[0], "Scab":px.colors.qualitative.Plotly[0], "Rust":px.colors.qualitative.Plotly[0], "Multiple diseases":px.colors.qualitative.Plotly[0]}
if list.index(preds.tolist(), max(preds)) == 0:
    pred = "Healthy"
if list.index(preds.tolist(), max(preds)) == 1:
    pred = "Scab"
if list.index(preds.tolist(), max(preds)) == 2:
    pred = "Rust"
if list.index(preds.tolist(), max(preds)) == 3:
    pred = "Multiple diseases"

colors[pred] = px.colors.qualitative.Plotly[1]
colors["Healthy"] = "seagreen"
colors = [colors[val] for val in colors.keys()]
fig.add_trace(go.Image(z=cv2.resize(train_images[2], (205, 136))), row=1, col=1)
fig.add_trace(go.Bar(x=["Healthy", "Multiple diseases", "Rust", "Scab"], y=preds, marker=dict(color=colors)), row=1, col=2)
fig.update_layout(height=1200, width=800, title_text="DenseNet Predictions", showlegend=False)

preds = predict(train_images[0])
colors = {"Healthy":px.colors.qualitative.Plotly[0], "Scab":px.colors.qualitative.Plotly[0], "Rust":px.colors.qualitative.Plotly[0], "Multiple diseases":px.colors.qualitative.Plotly[0]}
if list.index(preds.tolist(), max(preds)) == 0:
    pred = "Healthy"
if list.index(preds.tolist(), max(preds)) == 1:
    pred = "Multiple diseases"
if list.index(preds.tolist(), max(preds)) == 2:
    pred = "Rust"
if list.index(preds.tolist(), max(preds)) == 3:
    pred = "Scab"
    
colors[pred] = px.colors.qualitative.Plotly[1]
colors["Multiple diseases"] = "seagreen"
colors = [colors[val] for val in colors.keys()]
fig.add_trace(go.Image(z=cv2.resize(train_images[0], (205, 136))), row=2, col=1)
fig.add_trace(go.Bar(x=["Healthy", "Multiple diseases", "Rust", "Scab"], y=preds, marker=dict(color=colors)), row=2, col=2)

preds = predict(train_images[3])
colors = {"Healthy":px.colors.qualitative.Plotly[0], "Scab":px.colors.qualitative.Plotly[0], "Rust":px.colors.qualitative.Plotly[0], "Multiple diseases":px.colors.qualitative.Plotly[0]}
if list.index(preds.tolist(), max(preds)) == 0:
    pred = "Healthy"
if list.index(preds.tolist(), max(preds)) == 1:
    pred = "Multiple diseases"
if list.index(preds.tolist(), max(preds)) == 2:
    pred = "Rust"
if list.index(preds.tolist(), max(preds)) == 3:
    pred = "Scab"
    
colors[pred] = px.colors.qualitative.Plotly[1]
colors["Rust"] = "seagreen"
colors = [colors[val] for val in colors.keys()]
fig.add_trace(go.Image(z=cv2.resize(train_images[3], (205, 136))), row=3, col=1)
fig.add_trace(go.Bar(x=["Healthy", "Multiple diseases", "Rust", "Scab"], y=preds, marker=dict(color=colors)), row=3, col=2)

preds = predict(train_images[1])
colors = {"Healthy":px.colors.qualitative.Plotly[0], "Scab":px.colors.qualitative.Plotly[0], "Rust":px.colors.qualitative.Plotly[0], "Multiple diseases":px.colors.qualitative.Plotly[0]}
if list.index(preds.tolist(), max(preds)) == 0:
    pred = "Healthy"
if list.index(preds.tolist(), max(preds)) == 1:
    pred = "Multiple diseases"
if list.index(preds.tolist(), max(preds)) == 2:
    pred = "Rust"
if list.index(preds.tolist(), max(preds)) == 3:
    pred = "Scab"
    
colors[pred] = px.colors.qualitative.Plotly[1]
colors["Scab"] = "seagreen"
colors = [colors[val] for val in colors.keys()]
fig.add_trace(go.Image(z=cv2.resize(train_images[1], (205, 136))), row=4, col=1)
fig.add_trace(go.Bar(x=["Healthy", "Multiple diseases", "Rust", "Scab"], y=preds, marker=dict(color=colors)), row=4, col=2)

fig.update_layout(template="plotly_white")

画像と何の病気に分類されたかが可視化できた。良い精度で学習できているのが分かる。

## EfficientNet <a id="3.3"></a>

EfficientNetはCNNベースのImageNetモデル。今回はこれを用いてモデルを構築する

In [ ]:
with strategy.scope():
    model = tf.keras.Sequential([efn.EfficientNetB7(input_shape=(512, 512, 3),
                                                    weights='imagenet',
                                                    include_top=False),
                                 L.GlobalAveragePooling2D(),
                                 L.Dense(train_labels.shape[1],
                                         activation='softmax')])
    
    
        
    model.compile(optimizer='adam',
                  loss = 'categorical_crossentropy',
                  metrics=['categorical_accuracy'])
    model.summary()

### EfficientNetの基本的な構造

In [ ]:
SVG(tf.keras.utils.model_to_dot(Model(model.layers[0].input, model.layers[0].layers[11].output), dpi=70).create(prog='dot', format='svg'))

EfficientNetの基本的な構造。

In [ ]:
SVG(tf.keras.utils.model_to_dot(model, dpi=70).create(prog='dot', format='svg'))

参考にしたカーネルと同じ流れでモデルを設計してみる。

### 訓練データの読み込み

In [ ]:
history = model.fit(train_dataset,
                    epochs=EPOCHS,
                    callbacks=[lr_schedule],
                    steps_per_epoch=STEPS_PER_EPOCH,
                    validation_data=valid_dataset)

### 学習結果の可視化

In [ ]:
## densenetと同じく学習精度の推移を可視化
display_training_curves(
    history.history['categorical_accuracy'], 
    history.history['val_categorical_accuracy'], 
    'accuracy')

trainもvalidもおおよそ学習が出来ている。特にvalidationの方がdensenetに比べて比較的早いエポックで90%近くの精度まで上がっている。

efficientnetの方が学習スピードは早いのか。

In [ ]:
## densenetと同じ関数を使用
def process(img):
    return cv2.resize(img/255.0, (512, 512)).reshape(-1, 512, 512, 3)
def predict(img):
    return model.layers[2](model.layers[1](model.layers[0](process(img)))).numpy()[0]

fig = make_subplots(rows=4, cols=2)
preds = predict(train_images[2])

colors = {"Healthy":px.colors.qualitative.Plotly[0], "Scab":px.colors.qualitative.Plotly[0], "Rust":px.colors.qualitative.Plotly[0], "Multiple diseases":px.colors.qualitative.Plotly[0]}
if list.index(preds.tolist(), max(preds)) == 0:
    pred = "Healthy"
if list.index(preds.tolist(), max(preds)) == 1:
    pred = "Scab"
if list.index(preds.tolist(), max(preds)) == 2:
    pred = "Rust"
if list.index(preds.tolist(), max(preds)) == 3:
    pred = "Multiple diseases"

colors[pred] = px.colors.qualitative.Plotly[1]
colors["Healthy"] = "seagreen"
colors = [colors[val] for val in colors.keys()]
fig.add_trace(go.Image(z=cv2.resize(train_images[2], (205, 136))), row=1, col=1)
fig.add_trace(go.Bar(x=["Healthy", "Multiple diseases", "Rust", "Scab"], y=preds, marker=dict(color=colors)), row=1, col=2)
fig.update_layout(height=1200, width=800, title_text="EfficientNet Predictions", showlegend=False)

preds = predict(train_images[0])
colors = {"Healthy":px.colors.qualitative.Plotly[0], "Scab":px.colors.qualitative.Plotly[0], "Rust":px.colors.qualitative.Plotly[0], "Multiple diseases":px.colors.qualitative.Plotly[0]}
if list.index(preds.tolist(), max(preds)) == 0:
    pred = "Healthy"
if list.index(preds.tolist(), max(preds)) == 1:
    pred = "Multiple diseases"
if list.index(preds.tolist(), max(preds)) == 2:
    pred = "Rust"
if list.index(preds.tolist(), max(preds)) == 3:
    pred = "Scab"
    
colors[pred] = px.colors.qualitative.Plotly[1]
colors["Multiple diseases"] = "seagreen"
colors = [colors[val] for val in colors.keys()]
fig.add_trace(go.Image(z=cv2.resize(train_images[0], (205, 136))), row=2, col=1)
fig.add_trace(go.Bar(x=["Healthy", "Multiple diseases", "Rust", "Scab"], y=preds, marker=dict(color=colors)), row=2, col=2)

preds = predict(train_images[3])
colors = {"Healthy":px.colors.qualitative.Plotly[0], "Scab":px.colors.qualitative.Plotly[0], "Rust":px.colors.qualitative.Plotly[0], "Multiple diseases":px.colors.qualitative.Plotly[0]}
if list.index(preds.tolist(), max(preds)) == 0:
    pred = "Healthy"
if list.index(preds.tolist(), max(preds)) == 1:
    pred = "Multiple diseases"
if list.index(preds.tolist(), max(preds)) == 2:
    pred = "Rust"
if list.index(preds.tolist(), max(preds)) == 3:
    pred = "Scab"
    
colors[pred] = px.colors.qualitative.Plotly[1]
colors["Rust"] = "seagreen"
colors = [colors[val] for val in colors.keys()]
fig.add_trace(go.Image(z=cv2.resize(train_images[3], (205, 136))), row=3, col=1)
fig.add_trace(go.Bar(x=["Healthy", "Multiple diseases", "Rust", "Scab"], y=preds, marker=dict(color=colors)), row=3, col=2)

preds = predict(train_images[1])
colors = {"Healthy":px.colors.qualitative.Plotly[0], "Scab":px.colors.qualitative.Plotly[0], "Rust":px.colors.qualitative.Plotly[0], "Multiple diseases":px.colors.qualitative.Plotly[0]}
if list.index(preds.tolist(), max(preds)) == 0:
    pred = "Healthy"
if list.index(preds.tolist(), max(preds)) == 1:
    pred = "Multiple diseases"
if list.index(preds.tolist(), max(preds)) == 2:
    pred = "Rust"
if list.index(preds.tolist(), max(preds)) == 3:
    pred = "Scab"
    
colors[pred] = px.colors.qualitative.Plotly[1]
colors["Scab"] = "seagreen"
colors = [colors[val] for val in colors.keys()]
fig.add_trace(go.Image(z=cv2.resize(train_images[1], (205, 136))), row=4, col=1)
fig.add_trace(go.Bar(x=["Healthy", "Multiple diseases", "Rust", "Scab"], y=preds, marker=dict(color=colors)), row=4, col=2)
fig.update_layout(template="plotly_white")

# まとめと考察 <a id="4"></a>

## 1. EDAについて
深層学習のため特徴量はこちらで用意していないが、多くのカーネルでrgbのチャネル値の分布をチェックしていたため、画像の色彩分布は学習の際に使われたのではないかと考える。

また、multi diseaseが画像数が他に比べて少なく精度への影響を懸念していたが、どちらのモデルでも問題なく学習が進んでいたため画像の傘増しが効いたのではないかと考えている。

## 2. 学習前の下準備について
エッジ検出をCanny関数を用いることで行う方法について学習した。仕組みが完全に理解できたわけではないが、光の勾配を算出してそこが大きい箇所が物体が変わった位置と捉えて、ほぼ正確な葉の位置を検出するというおおよその仕組みは理解できた。

## 3. 学習精度について
今回はdensenetとefficientnetの2つを用いて学習を行い、densenetよりもefficientnetの方がより少ないエポックで良い精度が出ていることが分かった。画像解析は事前に何を行うかではなく、どのようなモデルを適切な状況において使えるかが学習精度の鍵になってきそうなので、この辺のモデルの特徴はぜひ押さえておきたい。

## 今後の展望
今回学習の精度は良かったのだが、DenceNet・EfficientNet共にかなり計算時間がかかってしまった。学習したMobileNetを使うことで多少精度が下がったとしてもどれくらい計算コストが削減できるのかを調査してみたいと考えている。

また、今回の開発では汎化性能についてまではあまり言及できていないため、実際のkaggle等ではdropout等も用いながら汎化性能についても検証していきたい。